In [1]:
import cv2
import glob
from shutil import copyfile
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import pathlib

%matplotlib inline


In [2]:
emotions = ["neutral", "anger", "disgust", "happy", "surprise"]

In [3]:
train_data_dir = "ck_export/dataset/train"
test_data_dir = "ck_export/dataset/test"
def load_dataset(dataset_dir):
    images = []
    image_labels = []
    for index, emotion in enumerate(emotions):
        for image in glob.glob("%s/%s/*"% (dataset_dir, emotion)):
            image = cv2.imread(image)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            images.append(gray)
            image_labels.append(index)
    return images, image_labels

In [4]:
fishface = cv2.face.FisherFaceRecognizer_create()

def train_test_set(dataset_path):
    training_data, training_labels = load_dataset(dataset_path + "train")
    test_data, test_labels = load_dataset(dataset_path + "test")
    
    return np.array(training_data), training_labels, np.array(test_data), test_labels


def run_recognizer():
    training_data, training_labels, test_data, test_labels = train_test_set("./ck_export/dataset/")
    print("training fisher face classifier")
    print("size of training set is:", len(training_labels), "images")
    fishface.train(training_data, np.asarray(training_labels))
    
    print("predicting classification set")
    cnt=0
    correct=0
    incorrect=0
    for image in test_data:
        pred, conf = fishface.predict(image)
        if pred == test_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

In [5]:
test_data, test_labels = load_dataset(test_data_dir)

In [6]:
l = np.array(test_data)
print(l.shape)
# pred, conf = fishface.predict(l[10])

(217, 350, 350)


In [ ]:
correct = run_recognizer()
fishface.save("fisher face classifer.xml")
print("got", correct, "percent correct! ")

training fisher face classifier
size of training set is: 874 images
predicting classification set
got 98.15668202764977 percent correct! 


In [ ]:
faceDet = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt_tree.xml")

video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

feelings_faces = []
for index, emotion in enumerate(emotions):
  feelings_faces.append(cv2.imread('./emojis/' + emotion + '.png', -1))

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if frame is not None:
        # Predict result with network
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
#         gray = gray + 40
        
        #Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

        #Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            facefeatures = face
        elif len(face_two) == 1:
            facefeatures = face_two
        elif len(face_three) == 1:
            facefeatures = face_three
        elif len(face_four) == 1:
            facefeatures = face_four
        else:
            facefeatures = ""
        
        #Cut face
        
        for i, (x, y, w, h) in enumerate(facefeatures): #get coordinates and size of rectangle containing face
#             print("face found in file: %s" %f)
            gray = gray[y:y+h, x:x+w] #Cut the frame to size
            gray = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
            cv2.rectangle(frame, (x, y), (x + w, y+h), (0, 255, 0), 2)

        
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         gray = cv2.resize(gray, (350, 350))
            pred, conf = fishface.predict(gray)
        
            for index, emotion in enumerate(emotions):
                cv2.putText(frame, emotion, (10, index * 20 + 20), cv2.FONT_HERSHEY_PLAIN, 0.5, (0, 255, 0), 1);
#             cv2.rectangle(frame, (130, index * 20 + 10), (130 + int(pred * 100), (index + 1) * 20 + 4), (255, 0, 0), -1)
        
            face_image = feelings_faces[pred]

        # Ugly transparent fix
            for c in range(0, 3):
                frame[200 + (i*50):320, 10:130, c] = face_image[:,:,c] * (face_image[:, :, 3] / 255.0) +  frame[200:320, 10:130, c] * (1.0 - face_image[:, :, 3] / 255.0)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [21]:
video_capture.release()
cv2.destroyAllWindows()

In [ ]:

def brightness_script(dataset_dir):
    for index, emotion in enumerate(emotions):
        for image in glob.glob("%s/%s/*"% (dataset_dir, emotion)):
            image = cv2.imread(image)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            for i in range(10, 100)
    return images, image_labels